In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import math
import psycopg2 as psy
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

### Raw Data Source:

Raw data obtained from the National Youth in Transition Database (NYTD)

##### Starting datasets:

###### A) Services and demographic data for Fiscal Years (FY) 2011 - 2014 (Baseline populations for each FY)
        
- cross-sectional data collected over 8 six-month-periods

- for all foster youth receiving independent living services funded through CFCIP for each fiscal year

- data collected from all 50 states (my data excludes data from Connecticut) and D.C. and Puerto Rico

- Total of 675645 rows, and 31 columns
        
###### B) Outcomes data for Wave 1 and Wave 2 of Cohort 1
            
- cohort 1 consists of foster youth who received services in FY 2011 AND participated in surveys

- longitudinal data collected, 2 follow-up periods (every 2 years)

- about 5% of baseline population (Dataset A, FY 2011)

- Wave 1 = Outcomes Survey collected in FY 2011, within 45 days of youth's 17th birthday, from 49 states and D.C. and Puerto Rico (data excludes data from Connecticut) 

- Wave 2 = Outcomes Follow-Up Survey collected in FY 2013, within 45 days of youth's 19th birthday, from 48 states and D.C. (data excludes data from Connecticut, New York and Puerto Rico)

- Missing: Wave 3 = Outcomes Follow-Up Survey collected in FY 2015, within 45 days of youth's 21st birthday

- Total of 22811 rows, and 48 columns
        
###### C) Outcomes data for Wave 1 of Cohort 2
            
- cohort 2 consists of foster youth who received services in FY 2014 AND participated in surveys

- longitudinal data collected, 2 follow-up periods (every 2 years)

- about 5% of baseline population (Dataset A, FY 2014)

- data collected from 49 states and D.C. and Puerto Rico (data excludes data from Connecticut) 

- Wave 1 = Outcomes Survey collected in FY 2014, within 45 days of youth's 17th birthday

- Missing: Wave 2 = Outcomes Follow-Up Survey to be collected in FY 2016, within 45 days of youth's 21st birthday

- Missing: Wave 3 = Outcomes Follow-Up Survey to be collected in FY 2018, within 45 days of youth's 21st birthday

- Total of 23775 rows, and 49 columns

##### Data Cleaning and Munging Plan:
    
1) Load 3 starting datasets () into pandas dataframe for each

- As described in cell above: Dataset A, Dataset B, Dataset C

2) Create dataframe for Cohort 1 Dataset:

- Services and demographic data for foster youth in FY 2011 from dataset A 

- Outcomes data for Wave 1 and Wave 2 from dataset B

- Only include foster youth who completed surveys in both Wave 1 and Wave 2

- Clean data:

    - Drop rows that have missing values in multiple columns (>10)

    - Convert data type of columns AgeMP, EduLevlSv, RaceDcl, RaceUnkn to int

    - Convert data types of columns RepDates_outcomes, RepDates_services and DOB to datetime

    - Identify categorical variable columns that may need to be dummified for later analysis

    - Get rid of special characters (that are not UTF-8) in column HighEdCert

    - Unfortunately, several states have badly encoded unique ID values, which prevents me from tracking those 
      records. Need to drop data rows from the following states due to data quality issue:
      ["HI", "IN", "KY", "MS", "OR", "TX", "TN"]

- Load cleaned dataset into local postgres DB

3) Create dataframe for Cohort 2 Dataset:

- Services and demographic data for foster youth in FY 2014 from dataset A

- Outcomes data for Wave 1 from dataset C

- Only include foster youth who completed surveys in Wave 1

- Clean data:
    
    - Convert data type of columns AgeMP, EduLevlSv, RaceDcl, RaceUnkn to int

    - Convert data types of columns RepDates_outcomes, RepDates_services, outcmdte and DOB to datetime

    - Identify categorical variable columns that may need to be dummified for later analysis

    - Unfortunately, several states have badly encoded unique ID values, which prevents me from tracking those 
      records. Need to drop data rows from the following states due to data quality issue:
      ["HI", "IN", "KY", "MS", "OR", "TX", "TN"]

- Load cleaned dataset into local postgres DB

# START Data Clean and Munge

In [2]:
# Load 3 initial datasets into pandas DF

In [3]:
services_11_14 = pd.read_csv('capstone/data_to_use/raw/Services_2014.csv')

/Users/cguy/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,19,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
outcomes_2013 = pd.read_stata('capstone/data_to_use/raw/Outcomes2011Wave2_.dta')

In [5]:
outcomes_2014 = pd.read_stata('capstone/data_to_use/raw/Outcomes14_W1.dta')

In [6]:
list(services_11_14)

['FY',
 'RepDate',
 'StFIPS',
 'St',
 'RecNumbr',
 'DOB',
 'Sex',
 'AmIAKN',
 'Asian',
 'BlkAfrAm',
 'HawaiiPI',
 'White',
 'RaceUnkn',
 'RaceDcln',
 'HisOrgin',
 'FCStatSv',
 'LclFIPSsv',
 'TribeSv',
 'DelinqntSv',
 'EdLevlSv',
 'SpecEdSv',
 'ILNAsv',
 'AcSuppSv',
 'PSEdSuppSv',
 'CareerSv',
 'EmplyTrSv',
 'BudgetSv',
 'HousEdSv',
 'HlthEdSv',
 'FamSuppSv',
 'MentorSv',
 'SILsv',
 'RmBrdFASv',
 'EducFinaSv',
 'OthrFinaSv',
 'StFCID',
 'Race',
 'RaceEthn',
 'AgeMP']

In [7]:
list(outcomes_2013)

['Wave',
 'StChID',
 'StFIPS',
 'St',
 'RecNumbr',
 'RepDate',
 'dob',
 'Sex',
 'AmIAKN',
 'Asian',
 'BlkAfrAm',
 'HawaiiPI',
 'White',
 'RaceUnkn',
 'RaceDcln',
 'HisOrgin',
 'OutcmRpt',
 'OutcmDte',
 'OutcmFCS',
 'CurrFTE',
 'CurrPTE',
 'EmplySklls',
 'SocSecrty',
 'EducAid',
 'PubFinAs',
 'PubFoodAs',
 'PubHousAs',
 'OthrFinAs',
 'HighEdCert',
 'CurrenRoll',
 'CnctAdult',
 'Homeless',
 'SubAbuse',
 'Incarc',
 'Children',
 'Marriage',
 'Medicaid',
 'OthrHlthIn',
 'MedicalIn',
 'MentlHlthIn',
 'PrescripIn',
 'SampleState',
 'InSample',
 'Baseline',
 'FY11Cohort',
 'Elig19',
 'Weight',
 'Responded']

In [8]:
list(outcomes_2014)

['wave',
 'stfips',
 'st',
 'recnumbr',
 'repdate',
 'dob',
 'sex',
 'amiakn',
 'asian',
 'blkafram',
 'hawaiipi',
 'white',
 'raceunkn',
 'racedcln',
 'hisorgin',
 'outcmrpt',
 'outcmdte',
 'outcmfcs',
 'currfte',
 'currpte',
 'emplysklls',
 'socsecrty',
 'educaid',
 'pubfinas',
 'pubfoodas',
 'pubhousas',
 'othrfinas',
 'highedcert',
 'currenroll',
 'cnctadult',
 'homeless',
 'subabuse',
 'incarc',
 'children',
 'marriage',
 'medicaid',
 'othrhlthin',
 'medicalin',
 'mentlhlthin',
 'prescripin',
 'baseline',
 'fy14cohort',
 'elig19',
 'samplestate',
 'insample',
 'responded',
 'race',
 'raceethn',
 'stfcid']

In [9]:
# Create 2 datasets: Cohort 1 and Cohort 2

In [10]:
# For cohort 1, only need services data for FY 2011

cohort_1 = services_11_14[services_11_14.FY == 2011]
cohort_1 = cohort_1.drop(['RecNumbr','LclFIPSsv'],axis= 1)

# For now, I only want to examine data for foster youth who are in both W1 and W2

cohort_1.rename(columns={'RepDate':'RepDate_services'}, inplace=True)

outcomes_2013_keep = outcomes_2013.drop(['StFIPS','St','RecNumbr','dob','Sex','AmIAKN','Asian','BlkAfrAm',\
                                         'HawaiiPI','White','RaceUnkn','RaceDcln','HisOrgin'], axis=1)

outcomes_2013_keep.rename(columns={'RepDate':'RepDate_outcomes', 'StChID': 'StFCID'}, inplace=True)

cohort_1 = pd.merge(outcomes_2013_keep, cohort_1, on='StFCID', how='inner')

In [11]:
# Issue with encoding the tracking ID (StFIPS). Need to drop states that have the issue: HI, IN, KY, MS, OR, TX, TN
# bad_encode_states = ["HI", "IN", "KY", "MS", "OR", "TX", "TN"]
cohort_1 = cohort_1[cohort_1.St != "HI"]
cohort_1 = cohort_1[cohort_1.St != "IN"]
cohort_1 = cohort_1[cohort_1.St != "KY"]
cohort_1 = cohort_1[cohort_1.St != "MS"]
cohort_1 = cohort_1[cohort_1.St != "OR"]
cohort_1 = cohort_1[cohort_1.St != "TX"]
cohort_1 = cohort_1[cohort_1.St != "TN"]

dict(cohort_1.StFCID.value_counts())

{'VA269626272263': 4,
 'SC000617715839': 4,
 'AZ008000184681': 2,
 'OHGBOSK46DTHR9': 1,
 'NY260114088260': 2,
 'SC000837155133': 4,
 'MO207576976709': 2,
 'KS0A0A0B1C8G9H': 2,
 'AZ005000546394': 1,
 'DC223329222229': 1,
 'SC000787935532': 4,
 'CA00FlVV71l52F': 2,
 'CA00IF2F3Ce6nO': 2,
 'LAXXXCZSPSIGHX': 2,
 'KS0A0A0A7C1I2J': 2,
 'UT000027083094': 2,
 'WISMOVO0M6JE70': 4,
 'CA00KP2f7pp42A': 2,
 'IAN7ZNY0YHLQMS': 2,
 'CA00JZjVB8F5Us': 1,
 'RI860946899999': 2,
 'NE338049924444': 1,
 'IAN7ZNYM8NE400': 4,
 'WISMOVO4S8DN08': 1,
 'AZ006000590577': 2,
 'WA002D3MPW03G6': 2,
 'KS0A0A0B1E5I6F': 4,
 'CA00B7zKCbi196': 2,
 'CA00KcAibHl3Oi': 2,
 'NY242134966130': 2,
 'CA005zDCeud4md': 1,
 'CA00OJAhn397QN': 2,
 'NC979953338516': 2,
 'NY240646360130': 1,
 'WISMOVOD1UZZ4E': 2,
 'MD100458939149': 2,
 'CA00Eg0xb9u192': 2,
 'FL600004529801': 1,
 'NY240841928010': 2,
 'NY240938469200': 4,
 'CA00Hr2HSiI192': 2,
 'FL142502046101': 1,
 'WISMOVO0M6JE7I': 4,
 'CO000221813118': 2,
 'FL186771010179': 1,
 'GA007400

In [12]:
cohort_1.describe()

/Users/cguy/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Weight,FY,RepDate_services,StFIPS,Sex,AmIAKN,Asian,BlkAfrAm,HawaiiPI,White,...,HousEdSv,HlthEdSv,FamSuppSv,MentorSv,SILsv,RmBrdFASv,EducFinaSv,OthrFinaSv,Race,RaceEthn
count,19339.000000,34220.0,34220.000000,34220.000000,34220.000000,34220.000000,34220.000000,34220.000000,34220.000000,34220.000000,...,34220.000000,34220.000000,34220.000000,34220.000000,34220.000000,34220.000000,34220.000000,34220.000000,34220.000000,34220.000000
mean,2.584101,2011.0,201106.250731,24.739509,1.508299,0.036645,0.014757,0.346084,0.005202,0.545617,...,9.377323,9.390094,9.329018,9.230362,9.126008,9.129456,9.157598,9.212858,11.043688,4.829193
std,2.483794,0.0,2.989548,17.243140,0.499938,0.187892,0.120582,0.475727,0.071936,0.497922,...,24.704123,24.699530,24.721429,24.756442,24.792996,24.791796,24.781983,24.762608,28.712294,13.945162
min,0.700000,2011.0,201103.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,NaN,2011.0,201103.000000,6.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,NaN,2011.0,201109.000000,22.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000
75%,NaN,2011.0,201109.000000,36.000000,2.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000,6.000000
max,80.710000,2011.0,201109.000000,72.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,99.000000,99.000000


In [13]:
# Need to figure out how to treat missing values....

dict(cohort_1.isnull().sum())

{'AcSuppSv': 0,
 'AgeMP': 0,
 'AmIAKN': 0,
 'Asian': 0,
 'Baseline': 0,
 'BlkAfrAm': 0,
 'BudgetSv': 0,
 'CareerSv': 0,
 'Children': 154,
 'CnctAdult': 154,
 'CurrFTE': 154,
 'CurrPTE': 154,
 'CurrenRoll': 154,
 'DOB': 0,
 'DelinqntSv': 0,
 'EdLevlSv': 0,
 'EducAid': 154,
 'EducFinaSv': 0,
 'Elig19': 0,
 'EmplySklls': 154,
 'EmplyTrSv': 0,
 'FCStatSv': 0,
 'FY': 0,
 'FY11Cohort': 0,
 'FamSuppSv': 0,
 'HawaiiPI': 0,
 'HighEdCert': 200,
 'HisOrgin': 0,
 'HlthEdSv': 0,
 'Homeless': 154,
 'HousEdSv': 0,
 'ILNAsv': 0,
 'InSample': 18613,
 'Incarc': 154,
 'Marriage': 154,
 'Medicaid': 154,
 'MedicalIn': 154,
 'MentlHlthIn': 154,
 'MentorSv': 0,
 'OthrFinAs': 154,
 'OthrFinaSv': 0,
 'OthrHlthIn': 154,
 'OutcmDte': 4135,
 'OutcmFCS': 154,
 'OutcmRpt': 154,
 'PSEdSuppSv': 0,
 'PrescripIn': 154,
 'PubFinAs': 154,
 'PubFoodAs': 154,
 'PubHousAs': 157,
 'Race': 0,
 'RaceDcln': 0,
 'RaceEthn': 0,
 'RaceUnkn': 0,
 'RepDate_outcomes': 0,
 'RepDate_services': 0,
 'Responded': 0,
 'RmBrdFASv': 0,
 'SIL

In [14]:
# For now, I am dropping rows that have NaNs in 10+ columns....

cohort_1 = cohort_1[cohort_1.HighEdCert.notnull()]
cohort_1 = cohort_1[cohort_1.PubHousAs.notnull()]

In [15]:
dict(cohort_1.isnull().sum())

{'AcSuppSv': 0,
 'AgeMP': 0,
 'AmIAKN': 0,
 'Asian': 0,
 'Baseline': 0,
 'BlkAfrAm': 0,
 'BudgetSv': 0,
 'CareerSv': 0,
 'Children': 0,
 'CnctAdult': 0,
 'CurrFTE': 0,
 'CurrPTE': 0,
 'CurrenRoll': 0,
 'DOB': 0,
 'DelinqntSv': 0,
 'EdLevlSv': 0,
 'EducAid': 0,
 'EducFinaSv': 0,
 'Elig19': 0,
 'EmplySklls': 0,
 'EmplyTrSv': 0,
 'FCStatSv': 0,
 'FY': 0,
 'FY11Cohort': 0,
 'FamSuppSv': 0,
 'HawaiiPI': 0,
 'HighEdCert': 0,
 'HisOrgin': 0,
 'HlthEdSv': 0,
 'Homeless': 0,
 'HousEdSv': 0,
 'ILNAsv': 0,
 'InSample': 18569,
 'Incarc': 0,
 'Marriage': 0,
 'Medicaid': 0,
 'MedicalIn': 0,
 'MentlHlthIn': 0,
 'MentorSv': 0,
 'OthrFinAs': 0,
 'OthrFinaSv': 0,
 'OthrHlthIn': 0,
 'OutcmDte': 4135,
 'OutcmFCS': 0,
 'OutcmRpt': 0,
 'PSEdSuppSv': 0,
 'PrescripIn': 0,
 'PubFinAs': 0,
 'PubFoodAs': 0,
 'PubHousAs': 0,
 'Race': 0,
 'RaceDcln': 0,
 'RaceEthn': 0,
 'RaceUnkn': 0,
 'RepDate_outcomes': 0,
 'RepDate_services': 0,
 'Responded': 0,
 'RmBrdFASv': 0,
 'SILsv': 0,
 'SampleState': 0,
 'Sex': 0,
 'SocS

In [16]:
dict(cohort_1.dtypes)

{'AcSuppSv': dtype('int64'),
 'AgeMP': dtype('O'),
 'AmIAKN': dtype('int64'),
 'Asian': dtype('int64'),
 'Baseline': dtype('O'),
 'BlkAfrAm': dtype('int64'),
 'BudgetSv': dtype('int64'),
 'CareerSv': dtype('int64'),
 'Children': dtype('O'),
 'CnctAdult': dtype('O'),
 'CurrFTE': dtype('O'),
 'CurrPTE': dtype('O'),
 'CurrenRoll': dtype('O'),
 'DOB': dtype('O'),
 'DelinqntSv': dtype('int64'),
 'EdLevlSv': dtype('O'),
 'EducAid': dtype('O'),
 'EducFinaSv': dtype('int64'),
 'Elig19': dtype('O'),
 'EmplySklls': dtype('O'),
 'EmplyTrSv': dtype('int64'),
 'FCStatSv': dtype('int64'),
 'FY': dtype('int64'),
 'FY11Cohort': dtype('O'),
 'FamSuppSv': dtype('int64'),
 'HawaiiPI': dtype('int64'),
 'HighEdCert': dtype('O'),
 'HisOrgin': dtype('int64'),
 'HlthEdSv': dtype('int64'),
 'Homeless': dtype('O'),
 'HousEdSv': dtype('int64'),
 'ILNAsv': dtype('int64'),
 'InSample': dtype('O'),
 'Incarc': dtype('O'),
 'Marriage': dtype('O'),
 'Medicaid': dtype('O'),
 'MedicalIn': dtype('O'),
 'MentlHlthIn': dty

In [17]:
#Need to convert AgeMP, EduLevlSv to int

cohort_1['AgeMP'] = cohort_1['AgeMP'].apply(lambda x: int(x))
cohort_1['EdLevlSv'] = cohort_1['EdLevlSv'].apply(lambda x: int(x))

In [18]:
# Fix mixed types in RaceDcl, RaceUnkn (need to be type int):

def treat_blank_strings(row_with_string):
    """Find blank strings ("") and convert to value 77 (which represents blank in raw data codebook). 
    Also, convert all values to type int."""
    if type(row_with_string) == int:
        return row_with_string
    elif row_with_string == "0":
        return 0
    elif row_with_string == "1":
        return 1
    else:
        return 77

In [19]:
print cohort_1.RaceDcln.value_counts()
cohort_1['RaceDcln'] = cohort_1['RaceDcln'].apply(treat_blank_strings)
cohort_1.RaceDcln.value_counts()

0    16187
0    11466
      4026
1     1245
1     1093
Name: RaceDcln, dtype: int64


0     27653
77     4026
1      2338
Name: RaceDcln, dtype: int64

In [20]:
print cohort_1.RaceUnkn.value_counts()
cohort_1['RaceUnkn'] = cohort_1['RaceUnkn'].apply(treat_blank_strings)
cohort_1.RaceUnkn.value_counts()

0    30034
0     3022
       484
1      441
1       36
Name: RaceUnkn, dtype: int64


0     33056
77      484
1       477
Name: RaceUnkn, dtype: int64

In [21]:
#Need to convert RepDates and DOBs to datetime

cohort_1.RepDate_services = pd.to_datetime(cohort_1['RepDate_services'], format="%Y%m")
cohort_1.DOB = pd.to_datetime(cohort_1['DOB'])

In [22]:
# Need to address blanks in RepDates_outcomes column before converting to datetime....

print cohort_1.RepDate_outcomes.value_counts()

def treat_blank_dates(row_with_string):
    """Find blank dates ("") and convert to value 199901 (I need to decide what to do with blanks later)."""
    if len(row_with_string) == 0:
        return int("199901")
    else:
        return row_with_string

cohort_1['RepDate_outcomes'] = cohort_1['RepDate_outcomes'].apply(treat_blank_dates)
print cohort_1.RepDate_outcomes.value_counts()
cohort_1.RepDate_outcomes = pd.to_datetime(cohort_1['RepDate_outcomes'], format="%Y%m")

201103    11517
201109    10248
201309     5689
201303     5674
            543
201203      346
Name: RepDate_outcomes, dtype: int64
201103    11517
201109    10248
201309     5689
201303     5674
199901      543
201203      346
Name: RepDate_outcomes, dtype: int64


In [23]:
dict(cohort_1.dtypes)

{'AcSuppSv': dtype('int64'),
 'AgeMP': dtype('int64'),
 'AmIAKN': dtype('int64'),
 'Asian': dtype('int64'),
 'Baseline': dtype('O'),
 'BlkAfrAm': dtype('int64'),
 'BudgetSv': dtype('int64'),
 'CareerSv': dtype('int64'),
 'Children': dtype('O'),
 'CnctAdult': dtype('O'),
 'CurrFTE': dtype('O'),
 'CurrPTE': dtype('O'),
 'CurrenRoll': dtype('O'),
 'DOB': dtype('<M8[ns]'),
 'DelinqntSv': dtype('int64'),
 'EdLevlSv': dtype('int64'),
 'EducAid': dtype('O'),
 'EducFinaSv': dtype('int64'),
 'Elig19': dtype('O'),
 'EmplySklls': dtype('O'),
 'EmplyTrSv': dtype('int64'),
 'FCStatSv': dtype('int64'),
 'FY': dtype('int64'),
 'FY11Cohort': dtype('O'),
 'FamSuppSv': dtype('int64'),
 'HawaiiPI': dtype('int64'),
 'HighEdCert': dtype('O'),
 'HisOrgin': dtype('int64'),
 'HlthEdSv': dtype('int64'),
 'Homeless': dtype('O'),
 'HousEdSv': dtype('int64'),
 'ILNAsv': dtype('int64'),
 'InSample': dtype('O'),
 'Incarc': dtype('O'),
 'Marriage': dtype('O'),
 'Medicaid': dtype('O'),
 'MedicalIn': dtype('O'),
 'Men

In [24]:
# Depending on tyoe of modeling techniques I will use, 
# I will need to turn the following into numeric categories: 

# wave, OutcmRpt, OutcmFCS, CurrFTE, CurrPTE, EmplySkills, SocSecrty,
# 'EducAid','PubFinAs','PubFoodAs','PubHousAs','OthrFinAs','HighEdCert','CurrenRoll','CnctAdult',
# 'Marriage','Medicaid','OthrHlthIn','MedicalIn','MentlHlthIn','PrescripIn','Baseline','FY11Cohort',
# 'Elig19','SampleState','InSample','Responded'

In [25]:
# Need to fix encoding in column HighEdCert

cohort_1.HighEdCert.value_counts()

None of the above          15041
Blank                      13142
High school diploma/GED     4699
Declined                     734
78.0                         201
Vocational certificate       132
Vocational license            35
Higher degree                 17
Associate�s degree            10
Bachelor�s degree              6
Name: HighEdCert, dtype: int64

In [26]:
def change_weird_chars(row):
    if type(row) == str:
        if 'Bachelor' in row:
            return "Bachelor's Degree"
        elif 'Associate' in row:
            return "Associate's Degree"
        else:
            return row
    
cohort_1.HighEdCert = cohort_1['HighEdCert'].apply(change_weird_chars)

cohort_1.HighEdCert.value_counts()

None of the above          15041
Blank                      13142
High school diploma/GED     4699
Declined                     734
Vocational certificate       132
Vocational license            35
Higher degree                 17
Associate's Degree            10
Bachelor's Degree              6
Name: HighEdCert, dtype: int64

In [27]:
cohort_1.head()

,Wave,StFCID,RepDate_outcomes,OutcmRpt,OutcmDte,OutcmFCS,CurrFTE,CurrPTE,EmplySklls,SocSecrty,...,HlthEdSv,FamSuppSv,MentorSv,SILsv,RmBrdFASv,EducFinaSv,OthrFinaSv,Race,RaceEthn,AgeMP
0,Wave 1: Age 17 Baseline Survey,AK450290395006,2011-03-01,Youth participated,2010-10-08,"Yes, is in FC on Date",No,No,No,"Yes, is receiving SSI or SSDI payments",...,0,0,0,0,0,0,1,1,1,17
1,Wave 1: Age 17 Baseline Survey,AK450290395006,2011-03-01,Youth participated,2010-10-08,"Yes, is in FC on Date",No,No,No,"Yes, is receiving SSI or SSDI payments",...,0,0,0,0,0,0,1,1,1,17
2,Wave 2: Age 19 Followup,AK450290395006,2013-03-01,Youth participated,2012-12-28,"No, is not in FC on Date",No,No,No,No,...,0,0,0,0,0,0,1,1,1,17
3,Wave 2: Age 19 Followup,AK450290395006,2013-03-01,Youth participated,2012-12-28,"No, is not in FC on Date",No,No,No,No,...,0,0,0,0,0,0,1,1,1,17
4,Wave 1: Age 17 Baseline Survey,AK450448396586,2011-03-01,Youth participated,2010-12-28,"Yes, is in FC on Date",No,No,Yes,No,...,0,0,0,0,0,0,0,3,3,17


In [28]:
cohort_1.describe()

,Weight,FY,StFIPS,Sex,AmIAKN,Asian,BlkAfrAm,HawaiiPI,White,RaceUnkn,...,HlthEdSv,FamSuppSv,MentorSv,SILsv,RmBrdFASv,EducFinaSv,OthrFinaSv,Race,RaceEthn,AgeMP
count,19298.000000,34017.0,34017.000000,34017.000000,34017.000000,34017.000000,34017.000000,34017.000000,34017.000000,34017.000000,...,34017.000000,34017.000000,34017.000000,34017.000000,34017.000000,34017.000000,34017.000000,34017.000000,34017.000000,34017.000000
mean,2.586975,2011.0,24.546756,1.508099,0.036658,0.014846,0.347473,0.005174,0.545198,1.109592,...,9.441926,9.380663,9.284652,9.178617,9.182703,9.210542,9.265985,10.954934,4.819620,16.543552
std,2.485643,0.0,16.992185,0.499942,0.187924,0.120936,0.476175,0.071744,0.497960,9.118341,...,24.763954,24.785990,24.820182,24.857457,24.856030,24.846288,24.826781,28.588251,13.976082,0.519542
min,0.700000,2011.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,14.000000
25%,NaN,2011.0,6.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,16.000000
50%,NaN,2011.0,22.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,17.000000
75%,NaN,2011.0,36.000000,2.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000,6.000000,17.000000
max,80.710000,2011.0,72.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,77.000000,...,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,99.000000,99.000000,20.000000


In [29]:
# Now that I have dataset for cohort_1 in a decent state, I will load this into a local postgres DB for storage

In [30]:
engine = create_engine('postgresql://cguy@localhost:5432/nytd_clean_data')
cohort_1.to_sql('cohort_1', engine)

In [31]:
%load_ext sql
# %reload_ext sql

/Users/cguy/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/cguy/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [32]:
%%sql postgresql://cguy@localhost:5432/nytd_clean_data
        
SELECT * FROM cohort_1 LIMIT 5;

5 rows affected.


index,Wave,StFCID,RepDate_outcomes,OutcmRpt,OutcmDte,OutcmFCS,CurrFTE,CurrPTE,EmplySklls,SocSecrty,EducAid,PubFinAs,PubFoodAs,PubHousAs,OthrFinAs,HighEdCert,CurrenRoll,CnctAdult,Homeless,SubAbuse,Incarc,Children,Marriage,Medicaid,OthrHlthIn,MedicalIn,MentlHlthIn,PrescripIn,SampleState,InSample,Baseline,FY11Cohort,Elig19,Weight,Responded,FY,RepDate_services,StFIPS,St,DOB,Sex,AmIAKN,Asian,BlkAfrAm,HawaiiPI,White,RaceUnkn,RaceDcln,HisOrgin,FCStatSv,TribeSv,DelinqntSv,EdLevlSv,SpecEdSv,ILNAsv,AcSuppSv,PSEdSuppSv,CareerSv,EmplyTrSv,BudgetSv,HousEdSv,HlthEdSv,FamSuppSv,MentorSv,SILsv,RmBrdFASv,EducFinaSv,OthrFinaSv,Race,RaceEthn,AgeMP
0,Wave 1: Age 17 Baseline Survey,AK450290395006,2011-03-01 00:00:00,Youth participated,2010-10-08 00:00:00,"Yes, is in FC on Date",No,No,No,"Yes, is receiving SSI or SSDI payments",No,No,No,No,No,None of the above,Yes,Yes,No,Yes,No,No,No,Yes,No,Don't Know,Yes,Yes,No,None,Yes,Yes,Yes,1.26,Responded to Survey,2011,2011-09-01 00:00:00,2,AK,1993-10-15 00:00:00,2,0,0,0,0,1,0,0,0,1,0,0,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,17
1,Wave 1: Age 17 Baseline Survey,AK450290395006,2011-03-01 00:00:00,Youth participated,2010-10-08 00:00:00,"Yes, is in FC on Date",No,No,No,"Yes, is receiving SSI or SSDI payments",No,No,No,No,No,None of the above,Yes,Yes,No,Yes,No,No,No,Yes,No,Don't Know,Yes,Yes,No,None,Yes,Yes,Yes,1.26,Responded to Survey,2011,2011-03-01 00:00:00,2,AK,1993-10-15 00:00:00,2,0,0,0,0,1,0,0,0,1,0,0,77,77,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,17
2,Wave 2: Age 19 Followup,AK450290395006,2013-03-01 00:00:00,Youth participated,2012-12-28 00:00:00,"No, is not in FC on Date",No,No,No,No,No,No,Yes,No,No,None of the above,No,Yes,Yes,No,No,No,Not Applicable,Yes,No,Not Applicable,Not Applicable,Not Applicable,No,No,Yes,Yes,Yes,1.58,Responded to Survey,2011,2011-09-01 00:00:00,2,AK,1993-10-15 00:00:00,2,0,0,0,0,1,0,0,0,1,0,0,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,17
3,Wave 2: Age 19 Followup,AK450290395006,2013-03-01 00:00:00,Youth participated,2012-12-28 00:00:00,"No, is not in FC on Date",No,No,No,No,No,No,Yes,No,No,None of the above,No,Yes,Yes,No,No,No,Not Applicable,Yes,No,Not Applicable,Not Applicable,Not Applicable,No,No,Yes,Yes,Yes,1.58,Responded to Survey,2011,2011-03-01 00:00:00,2,AK,1993-10-15 00:00:00,2,0,0,0,0,1,0,0,0,1,0,0,77,77,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,17
4,Wave 1: Age 17 Baseline Survey,AK450448396586,2011-03-01 00:00:00,Youth participated,2010-12-28 00:00:00,"Yes, is in FC on Date",No,No,Yes,No,No,Yes,Yes,Yes,No,None of the above,Yes,Yes,No,No,No,No,Not Applicable,No,No,No,No,No,No,None,Yes,Yes,Yes,1.36,Responded to Survey,2011,2011-09-01 00:00:00,2,AK,1993-12-15 00:00:00,2,1,0,0,0,0,0,0,0,1,1,0,11,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,17


# Now Cohort 2

In [33]:
# For cohort 2, only need services data for FY 2014

cohort_2 = services_11_14[services_11_14.FY == 2014]
cohort_2 = cohort_2.drop(['RecNumbr','LclFIPSsv'],axis= 1)

# For now, I only want to examine data for foster youth who are in W1

cohort_2.rename(columns={'RepDate':'RepDate_services'}, inplace=True)

outcomes_2014_keep = outcomes_2014.drop(['stfips','st','recnumbr','dob','sex','amiakn','asian','blkafram',\
                                         'hawaiipi','white','raceunkn','racedcln','hisorgin','race','raceethn'], axis=1)

outcomes_2014_keep.rename(columns={'repdate':'RepDate_outcomes', 'stfcid': 'StFCID'}, inplace=True)

cohort_2 = pd.merge(outcomes_2014_keep, cohort_2, on='StFCID', how='inner')

In [34]:
# Issue with encoding the tracking ID (StFIPS). Need to drop states that have the issue: HI, IN, KY, MS, OR, TX, TN
# bad_encode_states = ["HI", "IN", "KY", "MS", "OR", "TX", "TN"]
cohort_2 = cohort_2[cohort_2.St != "HI"]
cohort_2 = cohort_2[cohort_2.St != "IN"]
cohort_2 = cohort_2[cohort_2.St != "KY"]
cohort_2 = cohort_2[cohort_2.St != "MS"]
cohort_2 = cohort_2[cohort_2.St != "OR"]
cohort_2 = cohort_2[cohort_2.St != "TX"]
cohort_2 = cohort_2[cohort_2.St != "TN"]

dict(cohort_2.StFCID.value_counts())

{'MD100450694537': 1,
 'KS0A0A0B2G2E1I': 1,
 'IAN7ZNYMPIW4LH': 1,
 'CA003pCp0Fx4wm': 1,
 'NY241689189020': 2,
 'CA0080UqO7JAhd': 2,
 'CA00ICZpZY37S9': 2,
 'WA002D3PXD55JI': 2,
 'LAXXXCZZIGXCAA': 2,
 'AZ003000931714': 1,
 'CA005VFguhO196': 2,
 'CA002mA9a8PC06': 2,
 'CA004i6FjUG7XZ': 1,
 'NY241624566030': 2,
 'CA00HuyN4GsDR5': 2,
 'IL000067021290': 1,
 'OK005922257221': 1,
 'NC979948199989': 1,
 'IAN7ZNY0YKB4CH': 2,
 'PAYKC6DLQHUERG': 2,
 'UT000029414495': 1,
 'CA005b5aFDm5mf': 2,
 'NE305134024444': 2,
 'NY251184704180': 2,
 'LAXXXCZGZCAZAS': 2,
 'NV200000203058': 1,
 'MI000044261489': 1,
 'CA004O8s9EF5EE': 2,
 'CA00OfoQ30k4tA': 2,
 'FL310007596201': 2,
 'LAXXXCZZPSHZSH': 2,
 'KS0A0A0B2G9G6H': 1,
 'MO204573099316': 1,
 'PAJKCTML8KVNRG': 2,
 'MD100458295199': 1,
 'WISMOVO4ZQ5WH8': 1,
 'PAYKIVMLQAU6RG': 1,
 'VA214720242201': 2,
 'OHGB7SKN6TFHP9': 2,
 'MO923567227702': 2,
 'LAXXXCZZSAHGGC': 2,
 'NY241733632050': 2,
 'PAYKY08LQJU6RG': 1,
 'CA00U2tCu97AS2': 2,
 'MD100458238645': 1,
 'IAN7ZNY0

In [35]:
cohort_2.describe()

,responded,FY,RepDate_services,StFIPS,Sex,AmIAKN,Asian,BlkAfrAm,HawaiiPI,White,...,HousEdSv,HlthEdSv,FamSuppSv,MentorSv,SILsv,RmBrdFASv,EducFinaSv,OthrFinaSv,Race,RaceEthn
count,18253.000000,18253.0,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,...,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000
mean,0.781680,2014.0,201406.193448,23.266203,1.505122,0.047828,0.025201,0.362023,0.014409,0.560894,...,0.382458,0.395168,0.311620,0.231688,0.082562,0.103052,0.133622,0.246699,11.610146,3.912781
std,0.413117,0.0,2.993839,16.358002,0.499987,0.828726,0.815973,0.933927,0.809597,0.942099,...,1.469670,1.470632,1.462279,1.449742,0.629581,1.419919,1.428113,1.452440,29.305791,9.303250
min,0.000000,2014.0,201403.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,1.000000,2014.0,201403.000000,6.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,1.000000,2014.0,201409.000000,20.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000
75%,1.000000,2014.0,201409.000000,36.000000,2.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,6.000000
max,1.000000,2014.0,201409.000000,72.000000,2.000000,77.000000,77.000000,77.000000,77.000000,77.000000,...,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,99.000000,99.000000


In [36]:
# No NaNs....

dict(cohort_2.isnull().sum())

{'AcSuppSv': 0,
 'AgeMP': 0,
 'AmIAKN': 0,
 'Asian': 0,
 'BlkAfrAm': 0,
 'BudgetSv': 0,
 'CareerSv': 0,
 'DOB': 0,
 'DelinqntSv': 0,
 'EdLevlSv': 0,
 'EducFinaSv': 0,
 'EmplyTrSv': 0,
 'FCStatSv': 0,
 'FY': 0,
 'FamSuppSv': 0,
 'HawaiiPI': 0,
 'HisOrgin': 0,
 'HlthEdSv': 0,
 'HousEdSv': 0,
 'ILNAsv': 0,
 'MentorSv': 0,
 'OthrFinaSv': 0,
 'PSEdSuppSv': 0,
 'Race': 0,
 'RaceDcln': 0,
 'RaceEthn': 0,
 'RaceUnkn': 0,
 'RepDate_outcomes': 0,
 'RepDate_services': 0,
 'RmBrdFASv': 0,
 'SILsv': 0,
 'Sex': 0,
 'SpecEdSv': 0,
 'St': 0,
 'StFCID': 0,
 'StFIPS': 0,
 'TribeSv': 0,
 'White': 0,
 'baseline': 0,
 'children': 0,
 'cnctadult': 0,
 'currenroll': 0,
 'currfte': 0,
 'currpte': 0,
 'educaid': 0,
 'elig19': 0,
 'emplysklls': 0,
 'fy14cohort': 0,
 'highedcert': 0,
 'homeless': 0,
 'incarc': 0,
 'insample': 0,
 'marriage': 0,
 'medicaid': 0,
 'medicalin': 0,
 'mentlhlthin': 0,
 'othrfinas': 0,
 'othrhlthin': 0,
 'outcmdte': 0,
 'outcmfcs': 0,
 'outcmrpt': 0,
 'prescripin': 0,
 'pubfinas': 0,
 

In [37]:
dict(cohort_2.dtypes)

{'AcSuppSv': dtype('int64'),
 'AgeMP': dtype('O'),
 'AmIAKN': dtype('int64'),
 'Asian': dtype('int64'),
 'BlkAfrAm': dtype('int64'),
 'BudgetSv': dtype('int64'),
 'CareerSv': dtype('int64'),
 'DOB': dtype('O'),
 'DelinqntSv': dtype('int64'),
 'EdLevlSv': dtype('O'),
 'EducFinaSv': dtype('int64'),
 'EmplyTrSv': dtype('int64'),
 'FCStatSv': dtype('int64'),
 'FY': dtype('int64'),
 'FamSuppSv': dtype('int64'),
 'HawaiiPI': dtype('int64'),
 'HisOrgin': dtype('int64'),
 'HlthEdSv': dtype('int64'),
 'HousEdSv': dtype('int64'),
 'ILNAsv': dtype('int64'),
 'MentorSv': dtype('int64'),
 'OthrFinaSv': dtype('int64'),
 'PSEdSuppSv': dtype('int64'),
 'Race': dtype('int64'),
 'RaceDcln': dtype('O'),
 'RaceEthn': dtype('int64'),
 'RaceUnkn': dtype('O'),
 'RepDate_outcomes': dtype('O'),
 'RepDate_services': dtype('int64'),
 'RmBrdFASv': dtype('int64'),
 'SILsv': dtype('int64'),
 'Sex': dtype('int64'),
 'SpecEdSv': dtype('int64'),
 'St': dtype('O'),
 'StFCID': dtype('O'),
 'StFIPS': dtype('int64'),
 'Tr

In [38]:
#Need to convert AgeMP, EduLevlSv, RaceDcln, RaceUnkn to int

cohort_2['AgeMP'] = cohort_2['AgeMP'].apply(lambda x: int(x))
cohort_2['EdLevlSv'] = cohort_2['EdLevlSv'].apply(lambda x: int(x))
cohort_2['RaceDcln'] = cohort_2['RaceDcln'].apply(lambda x: int(x))
cohort_2['RaceUnkn'] = cohort_2['RaceUnkn'].apply(lambda x: int(x))


In [39]:
#Need to convert columns RepDates, DOB, and outcmdte to datetime

cohort_2['DOB'] = cohort_2['DOB'].apply(lambda x: x.zfill(10))

cohort_2.RepDate_outcomes = pd.to_datetime(cohort_2['RepDate_outcomes'], format="%Y%m")
cohort_2.RepDate_services = pd.to_datetime(cohort_2['RepDate_services'], format="%Y%m")
cohort_2.DOB = pd.to_datetime(cohort_2['DOB'])

In [40]:
cohort_2.DOB.value_counts()

1996-12-15    1792
1996-10-15    1682
1997-01-15    1640
1996-11-15    1634
1997-05-15    1557
1997-07-15    1495
1997-02-15    1490
1997-03-15    1472
1997-06-15    1412
1997-04-15    1388
1997-08-15    1353
1997-09-15    1332
1994-02-15       1
1996-08-15       1
1996-06-15       1
1997-10-15       1
1996-01-15       1
1992-08-15       1
Name: DOB, dtype: int64

In [41]:
# Need to address blanks in outcmdte column before converting to datetime....

print cohort_2.outcmdte.value_counts()

def treat_blank_dates2(row_with_string):
    """Find blank dates ("") and convert to value 01/01/1999 (I need to decide what to do with blanks later)."""
    if len(row_with_string) <= 3 :
        return "01/01/1999"
    else:
        return row_with_string

cohort_2['outcmdte'] = cohort_2['outcmdte'].apply(treat_blank_dates2)

print cohort_2.outcmdte.value_counts()

cohort_2.outcmdte = pd.to_datetime(cohort_2['outcmdte'], format="%m/%d/%Y")

              1323
1/22/2014      113
1/7/2014       113
3/24/2014      103
3/26/2014      102
3/31/2014      100
9/23/2014       98
1/14/2014       97
9/30/2014       97
11/14/2013      97
7/31/2014       95
3/25/2014       89
2/11/2014       89
1/2/2014        88
7/16/2014       88
7/2/2014        86
12/11/2013      86
9/24/2014       85
12/17/2013      85
1/29/2014       84
5/29/2014       84
4/23/2014       83
3/20/2014       83
8/6/2014        83
2/26/2014       82
8/19/2014       82
1/10/2014       81
1/9/2014        80
7/30/2014       80
11/27/2013      79
              ... 
4/12/2014        5
6/22/2014        4
10/2/2013        4
11/9/2013        4
10/27/2013       4
10/26/2014       4
10/25/2014       4
11/2/2014        4
8/24/2014        4
8/31/2014        4
8/23/2014        4
11/11/2014       4
11/5/2014        4
10/1/2013        4
10/19/2014       4
5/31/2014        3
11/1/2014        3
11/10/2014       3
11/18/2014       2
11/9/2014        2
3/1/2015         2
1/13/2013   

In [42]:
dict(cohort_2.dtypes)

{'AcSuppSv': dtype('int64'),
 'AgeMP': dtype('int64'),
 'AmIAKN': dtype('int64'),
 'Asian': dtype('int64'),
 'BlkAfrAm': dtype('int64'),
 'BudgetSv': dtype('int64'),
 'CareerSv': dtype('int64'),
 'DOB': dtype('<M8[ns]'),
 'DelinqntSv': dtype('int64'),
 'EdLevlSv': dtype('int64'),
 'EducFinaSv': dtype('int64'),
 'EmplyTrSv': dtype('int64'),
 'FCStatSv': dtype('int64'),
 'FY': dtype('int64'),
 'FamSuppSv': dtype('int64'),
 'HawaiiPI': dtype('int64'),
 'HisOrgin': dtype('int64'),
 'HlthEdSv': dtype('int64'),
 'HousEdSv': dtype('int64'),
 'ILNAsv': dtype('int64'),
 'MentorSv': dtype('int64'),
 'OthrFinaSv': dtype('int64'),
 'PSEdSuppSv': dtype('int64'),
 'Race': dtype('int64'),
 'RaceDcln': dtype('int64'),
 'RaceEthn': dtype('int64'),
 'RaceUnkn': dtype('int64'),
 'RepDate_outcomes': dtype('<M8[ns]'),
 'RepDate_services': dtype('<M8[ns]'),
 'RmBrdFASv': dtype('int64'),
 'SILsv': dtype('int64'),
 'Sex': dtype('int64'),
 'SpecEdSv': dtype('int64'),
 'St': dtype('O'),
 'StFCID': dtype('O'),
 

In [43]:
# Depending on tyoe of modeling techniques I will use, 
# I will need to turn the following into numeric categories: 

# wave, OutcmRpt, OutcmFCS, CurrFTE, CurrPTE, EmplySkills, SocSecrty,
# 'EducAid','PubFinAs','PubFoodAs','PubHousAs','OthrFinAs','HighEdCert','CurrenRoll','CnctAdult',
# 'Marriage','Medicaid','OthrHlthIn','MedicalIn','MentlHlthIn','PrescripIn','Baseline','FY11Cohort',
# 'Elig19','SampleState','InSample','Responded'

In [44]:
cohort_2.head()

,wave,RepDate_outcomes,outcmrpt,outcmdte,outcmfcs,currfte,currpte,emplysklls,socsecrty,educaid,...,HlthEdSv,FamSuppSv,MentorSv,SILsv,RmBrdFASv,EducFinaSv,OthrFinaSv,Race,RaceEthn,AgeMP
0,Age 17 Baseline Survey,2014-09-01,participated,2014-05-14,yes,no,yes,no,no,no,...,1,1,1,0,0,0,0,2,2,16
1,Age 17 Baseline Survey,2014-09-01,declined,1999-01-01,yes,blank,blank,blank,blank,blank,...,1,1,1,0,0,0,0,1,1,16
2,Age 17 Baseline Survey,2014-09-01,participated,2014-06-26,yes,no,no,no,no,no,...,1,1,1,0,0,0,0,1,7,16
3,Age 17 Baseline Survey,2014-09-01,participated,2014-09-29,yes,no,no,no,no,no,...,1,0,0,1,0,0,0,1,1,16
4,Age 17 Baseline Survey,2014-09-01,declined,1999-01-01,yes,blank,blank,blank,blank,blank,...,1,1,1,0,0,0,0,1,1,16


In [45]:
cohort_2.describe()

,responded,FY,StFIPS,Sex,AmIAKN,Asian,BlkAfrAm,HawaiiPI,White,RaceUnkn,...,HlthEdSv,FamSuppSv,MentorSv,SILsv,RmBrdFASv,EducFinaSv,OthrFinaSv,Race,RaceEthn,AgeMP
count,18253.000000,18253.0,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,...,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000,18253.000000
mean,0.781680,2014.0,23.266203,1.505122,0.047828,0.025201,0.362023,0.014409,0.560894,0.026023,...,0.395168,0.311620,0.231688,0.082562,0.103052,0.133622,0.246699,11.610146,3.912781,16.545992
std,0.413117,0.0,16.358002,0.499987,0.828726,0.815973,0.933927,0.809597,0.942099,0.816451,...,1.470632,1.462279,1.449742,0.629581,1.419919,1.428113,1.452440,29.305791,9.303250,0.499322
min,0.000000,2014.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,16.000000
25%,1.000000,2014.0,6.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,16.000000
50%,1.000000,2014.0,20.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,17.000000
75%,1.000000,2014.0,36.000000,2.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,6.000000,17.000000
max,1.000000,2014.0,72.000000,2.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,...,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,99.000000,99.000000,21.000000


In [46]:
# Now that I have dataset for cohort_1 in a decent state, I will load this into a local postgres DB for storage

In [47]:
cohort_2.to_sql('cohort_2', engine)

In [48]:
%%sql postgresql://cguy@localhost:5432/nytd_clean_data
        
SELECT * FROM cohort_2 LIMIT 5;

5 rows affected.


index,wave,RepDate_outcomes,outcmrpt,outcmdte,outcmfcs,currfte,currpte,emplysklls,socsecrty,educaid,pubfinas,pubfoodas,pubhousas,othrfinas,highedcert,currenroll,cnctadult,homeless,subabuse,incarc,children,marriage,medicaid,othrhlthin,medicalin,mentlhlthin,prescripin,baseline,fy14cohort,elig19,samplestate,insample,responded,StFCID,FY,RepDate_services,StFIPS,St,DOB,Sex,AmIAKN,Asian,BlkAfrAm,HawaiiPI,White,RaceUnkn,RaceDcln,HisOrgin,FCStatSv,TribeSv,DelinqntSv,EdLevlSv,SpecEdSv,ILNAsv,AcSuppSv,PSEdSuppSv,CareerSv,EmplyTrSv,BudgetSv,HousEdSv,HlthEdSv,FamSuppSv,MentorSv,SILsv,RmBrdFASv,EducFinaSv,OthrFinaSv,Race,RaceEthn,AgeMP
0,Age 17 Baseline Survey,2014-09-01 00:00:00,participated,2014-05-14 00:00:00,yes,no,yes,no,no,no,no,no,no,no,none of the above,yes,yes,no,no,no,no,not applicable,yes,no,not applicable,not applicable,not applicable,yes,yes,yes,no,Not a Sample State,1,AL000000227321,2014,2014-03-01 00:00:00,1,AL,1997-04-15 00:00:00,1,0,0,1,0,0,0,0,0,1,0,0,9,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0,2,2,16
1,Age 17 Baseline Survey,2014-09-01 00:00:00,declined,1999-01-01 00:00:00,yes,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,yes,no,no,no,Not a Sample State,0,AL000003716611,2014,2014-03-01 00:00:00,1,AL,1997-04-15 00:00:00,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,0,0,0,0,1,1,16
2,Age 17 Baseline Survey,2014-09-01 00:00:00,participated,2014-06-26 00:00:00,yes,no,no,no,no,no,no,no,no,no,none of the above,yes,yes,no,yes,no,no,not applicable,yes,no,not applicable,not applicable,not applicable,yes,yes,yes,no,Not a Sample State,1,AL000008395721,2014,2014-03-01 00:00:00,1,AL,1997-05-15 00:00:00,1,0,0,0,0,1,0,0,1,1,0,0,10,0,1,1,1,1,0,1,1,1,1,1,0,0,0,0,1,7,16
3,Age 17 Baseline Survey,2014-09-01 00:00:00,participated,2014-09-29 00:00:00,yes,no,no,no,no,no,no,no,no,no,none of the above,yes,yes,no,yes,no,no,not applicable,yes,no,not applicable,not applicable,not applicable,yes,yes,yes,no,Not a Sample State,1,AL000026713423,2014,2014-03-01 00:00:00,1,AL,1997-09-15 00:00:00,2,0,0,0,0,1,0,0,2,1,0,0,7,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,1,16
4,Age 17 Baseline Survey,2014-09-01 00:00:00,declined,1999-01-01 00:00:00,yes,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank,yes,no,no,no,Not a Sample State,0,AL000030924123,2014,2014-03-01 00:00:00,1,AL,1997-05-15 00:00:00,2,0,0,0,0,1,0,0,0,1,0,0,10,1,0,1,0,1,0,1,1,1,1,1,0,0,0,0,1,1,16
